# 第2章: SciPyによる線形計画法

この章では、SciPyを使って線形計画問題を解く方法を学びます。

In [1]:
from scipy.optimize import linprog

## 2.1 SciPyのlinprog()の基本

SciPyの `linprog()` 関数は、以下の形式の線形計画問題を解きます：

**最小化:** $c^T \cdot x$

**制約条件:**
- $A_{ub} \cdot x \leq b_{ub}$（不等式制約）
- $A_{eq} \cdot x = b_{eq}$（等式制約）
- $lb \leq x \leq ub$（変数の範囲）

### 重要な注意点

1. **linprog()は「最小化」のみ対応**
   - 最大化したい場合は、目的関数の係数に-1を掛ける

2. **不等式は「<=」のみ対応**
   - 「>=」の場合は、両辺に-1を掛けて「<=」に変換

3. **変数のデフォルト範囲は [0, +∞)**

## 例題1: 基本的な最小化問題

### 問題

**最小化:** $z = 3x + 2y$

**制約条件:**
- $x + y \geq 4$
- $2x + y \geq 6$
- $x \geq 0, y \geq 0$

### 解法

不等式「>=」を「<=」に変換するため、両辺に-1を掛けます：
- $-x - y \leq -4$
- $-2x - y \leq -6$

In [2]:
# 目的関数の係数（最小化なのでそのまま）
c = [3, 2]

# 不等式制約の左辺係数（>=を<=に変換するため-1を掛ける）
A_ub = [
    [-1, -1],  # -x - y <= -4
    [-2, -1],  # -2x - y <= -6
]

# 不等式制約の右辺
b_ub = [-4, -6]

# 最適化実行
result = linprog(c, A_ub=A_ub, b_ub=b_ub, method="highs")

print("【結果】")
print(f"ステータス: {result.message}")
print(f"成功: {result.success}")
print(f"最小値: {result.fun:.4f}")
print(f"x = {result.x[0]:.4f}")
print(f"y = {result.x[1]:.4f}")

【結果】
ステータス: Optimization terminated successfully. (HiGHS Status 7: Optimal)
成功: True
最小値: 10.0000
x = 2.0000
y = 2.0000


## 例題2: 最大化問題

### 問題

**最大化:** $z = x + 2y$

**制約条件:**
- $2x + y \leq 20$
- $-4x + 5y \leq 10$
- $-x + 2y \geq -2$
- $x \geq 0, y \geq 0$

### 解法

1. **最大化 → 最小化**: 目的関数の係数に-1を掛ける
   - 最小化: $z' = -x - 2y$

2. **>=を<=に変換**:
   - $-x + 2y \geq -2$ → $x - 2y \leq 2$

In [3]:
# 目的関数の係数（最大化なので-1を掛ける）
c = [-1, -2]

# 不等式制約の左辺係数
A_ub = [
    [2, 1],   # 2x + y <= 20
    [-4, 5],  # -4x + 5y <= 10
    [1, -2],  # x - 2y <= 2 （変換後）
]

# 不等式制約の右辺
b_ub = [20, 10, 2]

# 最適化実行
result = linprog(c, A_ub=A_ub, b_ub=b_ub, method="highs")

print("【結果】")
print(f"ステータス: {result.message}")
print(f"最大値: {-result.fun:.4f}")  # 符号を戻す
print(f"x = {result.x[0]:.4f}")
print(f"y = {result.x[1]:.4f}")

print("\n【解釈】")
print("最適解は x≈6.43, y≈7.14 で、最大値は約20.71です。")

【結果】
ステータス: Optimization terminated successfully. (HiGHS Status 7: Optimal)
最大値: 20.7143
x = 6.4286
y = 7.1429

【解釈】
最適解は x≈6.43, y≈7.14 で、最大値は約20.71です。


## 例題3: 等式制約を含む問題

### 問題

**最大化:** $z = x + 2y$

**制約条件:**
- $2x + y \leq 20$
- $-4x + 5y \leq 10$
- $-x + 2y \geq -2$
- $-x + 5y = 15$（等式制約）
- $x \geq 0, y \geq 0$

In [4]:
# 目的関数の係数
c = [-1, -2]

# 不等式制約
A_ub = [
    [2, 1],
    [-4, 5],
    [1, -2],
]
b_ub = [20, 10, 2]

# 等式制約
A_eq = [[-1, 5]]  # -x + 5y = 15
b_eq = [15]

# 最適化実行
result = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, method="highs")

print("【結果】")
print(f"ステータス: {result.message}")
print(f"最大値: {-result.fun:.4f}")
print(f"x = {result.x[0]:.4f}")
print(f"y = {result.x[1]:.4f}")

print("\n【解釈】")
print("等式制約を追加したことで、実行可能領域が線分に制限されました。")
print("最適解は x≈7.73, y≈4.55 で、最大値は約16.82です。")

【結果】
ステータス: Optimization terminated successfully. (HiGHS Status 7: Optimal)
最大値: 16.8182
x = 7.7273
y = 4.5455

【解釈】
等式制約を追加したことで、実行可能領域が線分に制限されました。
最適解は x≈7.73, y≈4.55 で、最大値は約16.82です。


## 例題4: リソース配分問題（工場の生産計画）

### 問題

ある工場で4種類の製品を生産しています。各製品の利益と資源消費量は以下の通りです：

|  | 利益 | 人員 | 原材料A | 原材料B |
|--|------|------|---------|--------|
| 製品1 | $20 | 1 | 3 | 0 |
| 製品2 | $12 | 1 | 2 | 1 |
| 製品3 | $40 | 1 | 1 | 2 |
| 製品4 | $25 | 1 | 0 | 3 |

**制約：**
- 1日の総生産量は50個以下（人員制約）
- 原材料Aは100単位まで使用可能
- 原材料Bは90単位まで使用可能

**目標:** 利益を最大化する生産量を求める

In [5]:
# 目的関数の係数（最大化なので-1を掛ける）
c = [-20, -12, -40, -25]

# 不等式制約の左辺係数
A_ub = [
    [1, 1, 1, 1],  # 人員制約
    [3, 2, 1, 0],  # 原材料A
    [0, 1, 2, 3],  # 原材料B
]

# 不等式制約の右辺
b_ub = [50, 100, 90]

# 最適化実行
result = linprog(c, A_ub=A_ub, b_ub=b_ub, method="highs")

print("【結果】")
print(f"ステータス: {result.message}")
print(f"最大利益: ${-result.fun:.2f}")

print("\n生産計画:")
products = ["製品1", "製品2", "製品3", "製品4"]
for name, val in zip(products, result.x):
    print(f"  {name}: {val:.2f} 個")

print("\n資源使用状況:")
resources = ["人員", "原材料A", "原材料B"]
for name, slack, limit in zip(resources, result.slack, b_ub):
    used = limit - slack
    print(f"  {name}: {used:.2f} / {limit} (余り: {slack:.2f})")

print("\n【解釈】")
print("最適解は製品1を5個、製品3を45個生産することです。")
print("製品2と製品4は利益率が低いため生産しません。")
print("原材料Bがボトルネックとなっています（全量使用）。")

【結果】
ステータス: Optimization terminated successfully. (HiGHS Status 7: Optimal)
最大利益: $1900.00

生産計画:
  製品1: 5.00 個
  製品2: 0.00 個
  製品3: 45.00 個
  製品4: 0.00 個

資源使用状況:
  人員: 50.00 / 50 (余り: 0.00)
  原材料A: 60.00 / 100 (余り: 40.00)
  原材料B: 90.00 / 90 (余り: 0.00)

【解釈】
最適解は製品1を5個、製品3を45個生産することです。
製品2と製品4は利益率が低いため生産しません。
原材料Bがボトルネックとなっています（全量使用）。


## 例題5: 変数の範囲制約

### 問題

**最小化:** $z = 2x + 3y$

**制約条件:**
- $x + y \leq 10$
- $2 \leq x \leq 8$（xの範囲）
- $1 \leq y \leq 6$（yの範囲）

In [6]:
# 目的関数の係数
c = [2, 3]

# 不等式制約
A_ub = [[1, 1]]
b_ub = [10]

# 変数の範囲（下限, 上限）
bounds = [
    (2, 8),  # 2 <= x <= 8
    (1, 6),  # 1 <= y <= 6
]

# 最適化実行
result = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method="highs")

print("【結果】")
print(f"ステータス: {result.message}")
print(f"最小値: {result.fun:.4f}")
print(f"x = {result.x[0]:.4f}")
print(f"y = {result.x[1]:.4f}")

print("\n【解釈】")
print("最小値は x=2, y=1 のとき z=7 です。")

【結果】
ステータス: Optimization terminated successfully. (HiGHS Status 7: Optimal)
最小値: 7.0000
x = 2.0000
y = 1.0000

【解釈】
最小値は x=2, y=1 のとき z=7 です。


## 例題6: 輸送問題（簡易版）

### 問題

2つの工場（A, B）から2つの店舗（1, 2）へ商品を輸送します。

**輸送コスト（$/個）:**

|  | 店舗1 | 店舗2 |
|--|-------|-------|
| 工場A | 4 | 6 |
| 工場B | 5 | 3 |

- 供給量: 工場A=50, 工場B=40
- 需要量: 店舗1=30, 店舗2=60

**決定変数:**
- x1: 工場A→店舗1
- x2: 工場A→店舗2
- x3: 工場B→店舗1
- x4: 工場B→店舗2

**目標:** 総輸送コストを最小化

In [7]:
# 目的関数の係数（輸送コスト）
c = [4, 6, 5, 3]

# 等式制約（供給量と需要量のバランス）
A_eq = [
    [1, 1, 0, 0],  # 工場Aからの出荷量 = 50
    [0, 0, 1, 1],  # 工場Bからの出荷量 = 40
    [1, 0, 1, 0],  # 店舗1への入荷量 = 30
    [0, 1, 0, 1],  # 店舗2への入荷量 = 60
]
b_eq = [50, 40, 30, 60]

# 最適化実行
result = linprog(c, A_eq=A_eq, b_eq=b_eq, method="highs")

print("【結果】")
print(f"ステータス: {result.message}")
print(f"最小輸送コスト: ${result.fun:.2f}")

print("\n輸送計画:")
routes = ["工場A→店舗1", "工場A→店舗2", "工場B→店舗1", "工場B→店舗2"]
for route, val in zip(routes, result.x):
    print(f"  {route}: {val:.0f} 個")

【結果】
ステータス: Optimization terminated successfully. (HiGHS Status 7: Optimal)
最小輸送コスト: $360.00

輸送計画:
  工場A→店舗1: 30 個
  工場A→店舗2: 20 個
  工場B→店舗1: 0 個
  工場B→店舗2: 40 個


## 第2章のまとめ

この章で学んだこと：

1. **scipy.optimize.linprog()** の基本的な使い方

2. **最大化問題の扱い方**
   - 目的関数の係数に-1を掛けて最小化問題に変換

3. **不等式「>=」の扱い方**
   - 両辺に-1を掛けて「<=」に変換

4. **等式制約の追加方法**
   - `A_eq`, `b_eq` パラメータを使用

5. **変数の範囲制約**
   - `bounds` パラメータを使用

### SciPyの制限

- 整数変数を扱えない
- 大規模問題には不向き
- 問題定義がやや面倒

---

**次の章:** より柔軟なPuLPライブラリを学びます！